In [2]:
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

## Model 1 - Basic CNN model

In [2]:
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict

data_batch_1 = unpickle('data_batch_1')
data_batch_2 = unpickle('data_batch_2')
data_batch_3 = unpickle('data_batch_3')
data_batch_4 = unpickle('data_batch_4')
data_batch_5 = unpickle('data_batch_5')
train_data = np.concatenate((data_batch_1[b'data'],data_batch_2[b'data'],data_batch_3[b'data'],data_batch_4[b'data'],data_batch_5[b'data']))
train_data = train_data.reshape((-1,3, 32, 32))
train_label = np.concatenate((data_batch_1[b'labels'],data_batch_2[b'labels'],data_batch_3[b'labels'],data_batch_4[b'labels'],data_batch_5[b'labels']))
print(train_data.shape, train_label.shape)

(50000, 3, 32, 32) (50000,)


In [46]:
class ImageDataset(Dataset):
    def __init__(self, datas, labels):
        self.datas = datas
        self.labels = labels

    def __len__(self):
        return len(self.datas)
    
    def __getitem__(self, idx):
        image = self.datas[idx]
        image = torch.tensor(image).float()
        label = self.labels[idx]
        label = torch.tensor(label).long()
        return image, label

# Define data transformations
transform = transforms.Compose([
    transforms.ToTensor()  # Convert PIL Image to PyTorch tensor
])

# Create an instance of ImageDataset with your data and transformations
train_dataset = ImageDataset(train_data, train_label)
train_dataloader = DataLoader(train_dataset, batch_size = 10, shuffle= True)

In [47]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.feature_extractor = nn.Sequential(
            nn.Conv2d(3,32, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Conv2d(32, 64, kernel_size = 3, padding = 1),
            nn.ELU(),
            nn.MaxPool2d(kernel_size = 2),
            nn.Flatten(),
        )
        self.classifier = nn.Linear(64*8*8, 10)
                    
    def forward(self, x):
        x = self.feature_extractor(x)
        x = self.classifier(x)
        return x

In [48]:
net = Net()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr = 0.001)

for epoch in range(1):
    for features, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [49]:
loss.item()

3.2135708332061768

## Model 2 - Pretrained model

In [20]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_train)

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])
test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)

train_loader = DataLoader(train_dataset, batch_size = 32, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 32, shuffle = True)
i,j = next(iter(train_loader))
print(i.shape, j.shape)


Files already downloaded and verified
Files already downloaded and verified
torch.Size([32, 3, 32, 32]) torch.Size([32])


In [11]:
from torchvision.models import resnet18, ResNet18_Weights

class MultiClass(nn.Module):
    def __init__(self):
        super(MultiClass, self).__init__()
        self.backbone = resnet18(weights = ResNet18_Weights.DEFAULT)
        self.classifier = nn.Sequential(
            nn.Linear(1000, 50),
            nn.ELU(),
            nn.Linear(50, 10),
            nn.Softmax()
        )
                    
    def forward(self, x):
        x = self.backbone(x)
        x = self.classifier(x)
        return x
MultiClass()

MultiClass(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track

1. parameters() and named_parameters(): Useful for examining and optimizing parameters.
2. .item(): Converts a one-element tensor to a scalar, commonly used for logging loss values.
3. state_dict() and load_state_dict(): Useful for saving and loading models.
4. train() and eval(): Switch the model between training and evaluation modes.

In [16]:
net = MultiClass()
for param in net.backbone.parameters():
    param.requires_grad = False
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(filter(lambda p: p.requires_grad, net.parameters()), lr=0.001)

for epoch in range(1):
    for features, labels in train_loader:
        optimizer.zero_grad()
        outputs = net(features)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()



In [17]:
loss.item()

2.2639403343200684

In [23]:
import torchmetrics
metric = torchmetrics.Accuracy(task = 'multiclass', num_classes = 10)
net.eval()
for features, labels in test_loader:
    outputs = net(features)
    acc = metric(outputs,labels)
acc = metric.compute()
print(acc)

c:\Users\voqua\AppData\Local\Programs\Python\Python310\lib\site-packages\torch\nn\modules\module.py:1511: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor(0.2240)
